In [ ]:
#!pip install opencv-python

In [ ]:
import cv2
import numpy as np
import scipy
from scipy.spatial import distance
#from scipy.misc import imread
from imageio import imread
import pickle as pickle
import random
import os
import matplotlib.pyplot as plt

In [ ]:
# Feature extractor
def extract_features(image_path, vector_size=32):
    image = imread(image_path, pilmode="RGB")
    try:
        # Using KAZE, cause SIFT, ORB and other was moved to additional module
        # which is adding addtional pain during install
        alg = cv2.KAZE_create()
        # Dinding image keypoints
        kps = alg.detect(image)
        # Getting first 32 of them. 
        # Number of keypoints is varies depend on image size and color pallet
        # Sorting them based on keypoint response value(bigger is better)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        # computing descriptors vector
        kps, dsc = alg.compute(image, kps)
        # Flatten all of them in one big vector - our feature vector
        dsc = dsc.flatten()
        # Making descriptor of same size
        # Descriptor vector size is 64
        needed_size = (vector_size * 64)
        if dsc.size < needed_size:
            # if we have less the 32 descriptors then just adding zeros at the
            # end of our feature vector
            dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
    except cv2.error as e:
        print ('Error: ', e)
        return None

    return dsc


def batch_extractor(images_path, pickled_db_path="all_features.pck"):
    print(images_path)
    folders = [os.path.join(images_path, p) for p in sorted(os.listdir(images_path))]
    image_folders = folders[:-1]
    image_files = []
    for dir_path in image_folders:
        image_files.append([os.path.join(dir_path, p) for p in sorted(os.listdir(dir_path))])

    result = {}
    for f in image_files:
        print ('Extracting features from image %s' % f)
        for image in f:
            name = image.split('/')[-1].lower()
            print('feature extraction, ', image, name)
            result[name] = extract_features(image)
    
    # saving all our feature vectors in pickled file
    with open(pickled_db_path, 'wb') as fp:
        pickle.dump(result, fp)

In [ ]:
import os
import random
import scipy
#from scipy.misc import imread
from imageio import imread

def run():
    images_path = 'C:/Users/User/Downloads/shell_dataset/dataset/train'
    print(images_path)
    folders = [os.path.join(images_path, p) for p in sorted(os.listdir(images_path))]
    #print (folders[:-1])
    image_folders = folders[:-1]
    #print (type(image_folders))
    image_files = []
    for dir_path in image_folders:
        image_files.append([os.path.join(dir_path, p) for p in sorted(os.listdir(dir_path))])
#     print(len(image_files))
    print(len(image_files[0]), image_files[0][0])#, image_files[0][54])
#     print(image_files[0])
    
    
    # getting 3 random images 
    sample = random.sample(image_files[0], 3)
    
    batch_extractor(images_path)
    
run()